In [3]:
import pandas as pd

df = pd.read_csv("grosswendt_raw.csv")
df.head()

,chromosome,start,end,interaction ID,alignment score of target part,strand,miRNA ID,miRNA sequence,target sequence,number of reads,number of unique reads,read IDs,read sequences
0,chr2,187627970,187627998,human08246,32,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AATGTTACTATTACGTCTTCTTCTGCCT,6,6,"@ago2_kishore-hc_3834205_x1,@ago2_kishore-hc_4...","TGAGGTAGTAGGTTGTATAGTTTTATTACGTCTTCTTC,TGAGGTA..."
1,chr16,52382582,52382612,human04514,32,-,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AGAATCACACGTATGTGGTCTTCTGTGACT,1,1,@ago2_kishore-hc_1451499_x1,TGAGGTAGTAGGTTGTATAGACGTATGTGGTGTTCTGT
2,chr3,113530194,113530221,human11115,36,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AGGAAAGGTATTCTTCTATACAACTTG,1,1,@ago2_kishore-pc_4275663_x1,TGAGGTAGTAGGTTGTATAGACTCTTCTATACAACTTG
3,chrM,14360,14382,human08122,34,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,CACAGCACCAATCCTACCTCCA,1,1,@ago2_kishore-hc_959688_x1,AGAGGTAGTAGGTTGTATAGCACCAATCCTACCTCCA
4,chr13,29062126,29062156,human08660,31,-,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,CACGTTGGTCTCATGCCGCCTTCTGCTGGG,1,1,@ago2_kishore-hc_2120565_x1,TGAGGTAGTAGGTTGTATAGTCTCATACCGCCTTCTGC


In [4]:
# dropping irrelevant columns
cols_to_drop = [
    "alignment score of target part",
    "number of reads",
    "number of unique reads",
    "read IDs",
    "read sequences"
]
df = df.drop(columns=cols_to_drop)

df["target_length"] = df["end"] - df["start"]
df["mirna_length"] = df["miRNA sequence"].apply(len)
df["dataset"] = "grosswendt"

# adding id column
df["id"] = df["chromosome"] + "_" + df["start"].astype(str) + "_" + df["end"].astype(str) + "_" + df["strand"].astype(str) + "_" + df["miRNA ID"].astype(str).str[4:]
df.drop(columns=["interaction ID"], inplace=True)

rename_dict = {
    "chromosome": "chromosome",
    "miRNA ID": "mirna_name",
    "miRNA sequence": "mirna_seq",
    "target sequence": "target_seq",
    "target_length": "target_len",
    "dataset": "dataset"
}

df.rename(columns=rename_dict, inplace=True)

# moving the id column to the start of the dataframe
df = pd.concat([df.iloc[:, -1:], df.iloc[:, :-1]], axis=1)

df.head()

,id,chromosome,start,end,strand,mirna_name,mirna_seq,target_seq,target_len,mirna_length,dataset
0,chr2_187627970_187627998_+_let-7a,chr2,187627970,187627998,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AATGTTACTATTACGTCTTCTTCTGCCT,28,22,grosswendt
1,chr16_52382582_52382612_-_let-7a,chr16,52382582,52382612,-,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AGAATCACACGTATGTGGTCTTCTGTGACT,30,22,grosswendt
2,chr3_113530194_113530221_+_let-7a,chr3,113530194,113530221,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,AGGAAAGGTATTCTTCTATACAACTTG,27,22,grosswendt
3,chrM_14360_14382_+_let-7a,chrM,14360,14382,+,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,CACAGCACCAATCCTACCTCCA,22,22,grosswendt
4,chr13_29062126_29062156_-_let-7a,chr13,29062126,29062156,-,hsa-let-7a,TGAGGTAGTAGGTTGTATAGTT,CACGTTGGTCTCATGCCGCCTTCTGCTGGG,30,22,grosswendt


In [5]:
df.to_csv("../../processed/grosswendt/grosswendt.csv", index=False)